In [30]:
import pandas as pd
import requests
import msal
from io import StringIO
from powerbiclient import Report
from powerbiclient.authentication import DeviceCodeLoginAuthentication

In [31]:
# Auth object for demo
auth = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code LKGPUGUZ8 to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


In [32]:
report = Report(group_id='ceb0d1d0-6226-4aef-b245-fac7f89bb52e', report_id='dc20da54-0dde-4b7e-aa9c-4435bb7c71be', auth=auth)

In [33]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

In [34]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

In [35]:
# Render report
report

Report()

In [36]:
# Get pages
def get_active_page(self):
    # Get list of pages
    pages = self.get_pages()
    active_page = {}
    for page in pages:
        
        # Get active page
        if page['isActive'] == True:
            active_page = page
            break
    return active_page

In [ ]:
active_page = get_active_page(report)
active_page_name = active_page['name']
visuals = report.visuals_on_page(active_page_name)
#export visuals var in json format

#keys 'title' and 'type' from the visuals
#visuals = [{k: v for k, v in visual.items() if k in ['title', 'type']} for visual in visuals]
visuals

In [ ]:
# Define the types of visuals to export data for
visual_types_to_export = ['barChart', 'lineChart', 'pieChart']

# Iterate over the visuals and export data for the specified types
for visual in visuals:
    if visual['type'] in visual_types_to_export:
        try:
            summarized_exported_data = report.export_visual_data(active_page_name,visual['name'], rows=20)
            data = StringIO(summarized_exported_data)
            # Load data into pandas DataFrame
            df = pd.read_csv(data, sep=",")
            # Store the DataFrame as a CSV file in the 'csv' folder
            csv_file_path = f'csv/{visual["type"]}_{report._model_name}.csv'
            df.to_csv(csv_file_path, index=False)
            print(f"Data for visual '{report._model_name}' of type '{visual['type']}' exported successfully.")
        except Exception as e:
            print(f"Could not export data for visual '{report._model_name}' of type '{visual['type']}': {e}")